## The following code is how to get the embeddings from watsonx.ai:

In [ ]:
from ibm_watsonx_ai import Credentials
from ibm_watsonx_ai.foundation_models import Embeddings
from ibm_watsonx_ai.metanames import EmbedTextParamsMetaNames as EmbedParams
from ibm_watsonx_ai.foundation_models.utils.enums import EmbeddingTypes

url = ""
api_key = ""
project_id = ""

embed_params = {
     EmbedParams.TRUNCATE_INPUT_TOKENS: 3,
     EmbedParams.RETURN_OPTIONS: {
     'input_text': True
     }
}

embedding = Embeddings(
    model_id= 'ibm/granite-embedding-278m-multilingual',
    params= embed_params,
    credentials= Credentials(
        api_key = api_key
        ,
        url = url),
    project_id = project_id
)

docs = [
    "Artificial intelligence was founded as an academic discipline in 1956.",
    "Alan Turing was the first person to conduct substantial research in AI.",
    "Born in Maida Vale, London, Turing was raised in southern England.",
]

embedding_vectors = embedding.embed_documents(texts=docs)
print(embedding_vectors)

[[-0.05385726, 0.03515281, -0.009785932, 0.045339704, 0.024955189, -0.00010500697, 0.0820554, 0.023513408, 0.034431864, 0.04530713, 0.020655409, 0.021754755, -0.024293521, -0.008545833, 0.023383446, 0.03257054, 0.049415905, -0.052480068, 0.042218637, -0.002501399, -0.011074143, 0.044578504, -0.049293797, 0.001656567, -0.025546774, -0.010300098, 0.020979049, 0.02064655, -0.027048446, 0.05082884, 0.08327804, -0.03730775, 0.057444405, 0.02234176, 0.0032625953, 0.0470855, 0.015375289, -0.00837622, 0.006464742, 0.019587677, -0.026047217, 0.039926972, 0.045081187, 0.02572127, -0.061400194, -0.004964488, 0.023332696, 0.057668466, 0.029385386, -0.02270469, -0.036106855, -0.0011284583, 0.02641716, -0.043358408, -0.043252315, -0.0016243948, 0.012517323, 0.026783062, 0.0050150817, 0.042300075, 0.024006467, 0.012877897, -0.002399401, -0.023663407, 0.02752454, -0.0008692686, 0.023597576, -0.039117184, 0.03340828, 0.057747416, -0.015688542, 0.006692475, 0.025959643, 0.067213744, -0.030917928, 0.0299

## The possible embedding models:

In [13]:
# PRINT dict of Enums
client.foundation_models.EmbeddingModels.show()

{'GRANITE_EMBEDDING_107M_MULTILINGUAL': 'ibm/granite-embedding-107m-multilingual', 'GRANITE_EMBEDDING_278M_MULTILINGUAL': 'ibm/granite-embedding-278m-multilingual', 'SLATE_125M_ENGLISH_RTRVR': 'ibm/slate-125m-english-rtrvr', 'SLATE_125M_ENGLISH_RTRVR_V2': 'ibm/slate-125m-english-rtrvr-v2', 'SLATE_30M_ENGLISH_RTRVR': 'ibm/slate-30m-english-rtrvr', 'SLATE_30M_ENGLISH_RTRVR_V2': 'ibm/slate-30m-english-rtrvr-v2', 'MULTILINGUAL_E5_LARGE': 'intfloat/multilingual-e5-large', 'ALL_MINILM_L12_V2': 'sentence-transformers/all-minilm-l12-v2', 'ALL_MINILM_L6_V2': 'sentence-transformers/all-minilm-l6-v2'}


## Embedding dimension:

In [17]:
print(f"\n The dimension: {len(embedding_vectors[0])}")


 The dimension: 768


## Milvus Collection Creating

In [29]:
from pymilvus import MilvusClient
import numpy as np

client = MilvusClient("./milvus_demo.db")
client.create_collection(
    collection_name="demo_collection",
    dimension=len(embedding_vectors[0])  # The vectors we will use in this demo has 384 dimensions
)

data = [ {"id": i, "vector": embedding_vectors[i], "text": docs[i], "subject": "history"} for i in range(len(embedding_vectors)) ]
res = client.insert(
    collection_name="demo_collection",
    data=data
)
print(res)

{'insert_count': 3, 'ids': [0, 1, 2]}


## Build instance with watsonx.ai model

In [ ]:
from ibm_watsonx_ai import Credentials
from ibm_watsonx_ai.foundation_models import ModelInference
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models.utils.enums import ModelTypes, DecodingMethods

# To display example params enter
GenParams().get_example_values()

generate_params = {
    GenParams.MAX_NEW_TOKENS: 50
}

model_inference = ModelInference(
    model_id='google/flan-ul2',
    params=generate_params,
    credentials=Credentials(
        api_key = api_key,
        url = url),
    project_id=project_id
)

## Get Knowledge

In [76]:
question = ["Where was Turing born?"]
vectors = embedding.embed_documents(texts=question)

res = client.search(
    collection_name="demo_collection",
    data=[vectors[0]],
    filter="subject == 'history'",
    limit=2,
    output_fields=["text"],
)

knowledge=""
for context in res[0]:
    knowledge = knowledge + context.get('entity').get('text') + "\n"
print(knowledge)


Alan Turing was the first person to conduct substantial research in AI.
Born in Maida Vale, London, Turing was raised in southern England.



## Get Answer

In [77]:
q = "Where was Turing born?"
context = "You are a helpful assistant. And here are some background information for the questions the user going to ask:"
q = context + knowledge + q
generated_text = model_inference.generate_text(prompt=q)
print(generated_text)

Maida Vale, London
